In [1]:
import tensorflow as tf

In [ ]:
from tensorflow.python.client import device_lib

tf.config.list_physical_devices('GPU')
device_lib.list_local_devices()

# Plan

1. run the CNN-RNN architecture from "Show and Tell" (copy the network  and download the trained weights)
 - test it on mscoc
 - try it with the brain data
2. add attention mechanism
3. Use transformer network

---

    To make it possible to train the network on a cluster the actual network will be written into a .py file, while this    notebook is used to analyse the results.
    
---